<a href="https://colab.research.google.com/github/RealBJr/sign-language-classifier/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1) Environment Setup

In [7]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader

# 2) Data

## 2.1) Create Datasets

Creation of dummy dataset to ensure to ensure that pipeline works:

In [4]:
class DummyImageDataset(Dataset):
    def __init__(self, num_samples=1000, num_classes=10, image_size=(3, 224, 224)):
        self.num_samples = num_samples
        self.num_classes = num_classes
        self.image_size = image_size

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        image = torch.randn(self.image_size)          # fake image
        label = torch.randint(0, self.num_classes, (1,)).item()  # fake class
        return image, label

## 2.2) Data Splitting

Dummy data splitting in training and validation steps, used for testing pipeline works:

In [5]:
dummy_train_dataset = DummyImageDataset(num_samples=500)
dummy_val_dataset   = DummyImageDataset(num_samples=100)

## 2.3) Data Loaders

Dummy data loaders used for testing pipeline works:

In [9]:
dummy_train_loader = DataLoader(dummy_train_dataset, batch_size=32, shuffle=True)
dummy_val_loader   = DataLoader(dummy_val_dataset, batch_size=32)

## 2.4) Data Preprocessing

# 3) Training Procedure

## 3.1) Load Models

With pretrained models, the accuracy will theoretically be better because the model doesn't have to learn certain characteristics(classes) all over again. Models available [here](https://docs.pytorch.org/vision/main/models.html).

In [11]:
model_resnet = models.resnet18(weights="IMAGENET1K_V1")
model_mobilenet = models.mobilenet_v2(weights="IMAGENET1K_V1")
model_vgg = models.vgg16(weights="IMAGENET1K_V1")

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


100%|██████████| 528M/528M [00:08<00:00, 68.1MB/s]


## 3.2) Classifier Head Replacement

## 3.3) Freeze Feature Extractor

## 3.4) Optimizer + Loss Setup

## 3.5) Forward Pass

## 3.6) Loss Calculation

## 3.7) Backpropagation

## 3.8) Weight Update Step

## 3.9) Validation Phase

## 3.10) Optimization

## 3.11) Save Trained Model

# 4) Model Evaluation & Analysis